## Ajuste fino eficiente de parâmetros (PEFT) com LoRA

 Realizar o treinamento de LLMs demanda um custo computacional significativo, pois é importante considerar que o ajuste fino completo requer memória muitas vezes maior do que o próprio modelo, devido à necessidade de armazenamento de diversos outros parâmetros durante o processo de ajuste. Além disso, é essencial alocar memória para estados do otimizador, gradientes, ativações futuras e memória temporária ao longo de todo o treinamento. Esses componentes adicionais podem exceder rapidamente a capacidade do hardware de consumo. Neste contexto, os Large Language Models (LLMs) como GPT-4 ou LLaMA, onde o pré-treinamento envolve conjuntos de dados massivos, o ponto de partida para o ajuste fino é uma rede neural com bilhões de pesos. O ajuste fino de tal modelo é quase impossível sem grandes quantidades de computação, e é aqui que entram LoRA e QLoRA! Essas técnicas democratizam o acesso à IA generativa e são essenciais para quem deseja personalizar um LLM pré-treinado de maneira econômica.




 

In [ ]:
!pip install transformers[torch]
!pip install pandas peft datasets
!pip install accelerate -U

In [3]:
from transformers import AutoTokenizer, AutoModelForPreTraining, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset
from peft import LoraConfig, TaskType, get_peft_model
import pandas as pd
import torch

In [20]:
#Configurando Lora logo:
peft_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM,
    inference_mode = False,
    r = 8,
    lora_alpha=16,
    lora_dropout=0.1
)

In [21]:
#chamando o modelo
model_name = 'bigscience/bloom-560m'
model = AutoModelForPreTraining.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
lora_model = get_peft_model(model, peft_config)

In [22]:
data_file = {"train": "./Dados/treino.csv", "test":'./Dados/teste.csv'}
dataSetCompleto = load_dataset("csv", data_files=data_file)

In [23]:
# Separando os dados de treino e teste
dataset_treino = dataSetCompleto['train']
dataset_teste = dataSetCompleto['test']

In [ ]:
#Realziando a Toikenização dos dados para passar para o trinamento
def tokenizar(dados):
    return tokenizer(dados["pergunta"], dados["resposta"], truncation=True, padding="longest")

train_dataset = dataset_treino.map(tokenizar, batched=True)
test_dataset = dataset_teste.map(tokenizar, batched=True)

In [25]:
# A função DataCollatorForLanguageModeling() 
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [26]:
def compute_metrics(eval_prediction):
  loss = eval_prediction.loss
  perplexity = torch.exp(torch.tensor(loss)).item()
  return {"perplexity": perplexity, "eval_loss": loss}

In [31]:
training_args = TrainingArguments(
    output_dir="./novaPasta", 
    overwrite_output_dir=True,
    num_train_epochs=3,
    eval_strategy='epoch',
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
)

In [32]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dataset_teste,
    data_collator=data_collator,
    compute_metrics=compute_metrics     
)

In [ ]:
trainer.train()

In [ ]:
#Salvando o modelo ajustado em uma pasta
output_model_dir = "./novaPasta"
model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)